In [ ]:
import sys #Module to access the computer
import os

In [ ]:
# Set paths
homey = os.getcwd()
redouxPath = os.path.join(homey, 'ForecastRedoux')
sqlPath = os.path.join(redouxPath, 'SQL')
rawDataPath = os.path.join(redouxPath, 'RawData')
AdditionalInfoPath = os.path.join(homey, 'AdditionalInfo')

In [ ]:
# Connect to FB API
sys.path.insert(0, 'Z:\Python projects\FishbowlAPITestProject')
import connecttest

In [ ]:
myresults = connecttest.create_connection(sqlPath, 'BOMQuery.txt')
myexcel = connecttest.makeexcelsheet(myresults)
connecttest.save_workbook(myexcel, rawDataPath, 'BOMs.xlsx')
myresults = connecttest.create_connection(sqlPath, 'PartQuery.txt')
myexcel = connecttest.makeexcelsheet(myresults)
connecttest.save_workbook(myexcel, rawDataPath, 'Parts.xlsx')
myresults = connecttest.create_connection(sqlPath, 'MOQueryRedouxAlternate.txt')
myexcel = connecttest.makeexcelsheet(myresults)
connecttest.save_workbook(myexcel, rawDataPath, 'MOs.xlsx')
myresults = connecttest.create_connection(sqlPath, 'POQueryAlternate.txt')
myexcel = connecttest.makeexcelsheet(myresults)
connecttest.save_workbook(myexcel, rawDataPath, 'POs.xlsx')
myresults = connecttest.create_connection(sqlPath, 'SOQueryAlternate.txt')
myexcel = connecttest.makeexcelsheet(myresults)
connecttest.save_workbook(myexcel, rawDataPath, 'SOs.xlsx')
myresults = connecttest.create_connection(sqlPath, 'INVQueryAlternate.txt')
myexcel = connecttest.makeexcelsheet(myresults)
connecttest.save_workbook(myexcel, rawDataPath, 'INVs.xlsx')
myresults = connecttest.create_connection(sqlPath, 'DescQuery.txt')
myexcel = connecttest.makeexcelsheet(myresults)
connecttest.save_workbook(myexcel, rawDataPath, 'Descs.xlsx')

In [ ]:
import pandas as pd
import os
import sys

In [ ]:
sys.path.insert(0, redouxPath) #Pull up the file with the forecast information

In [ ]:
import ForecastMain #Import the actual forecast python file

In [ ]:
ForecastMain.run_normal_forecast_tiers_v2(
    ignore_schedule_errors=True, 
    add_stock_builds=False, 
    ignore_orders=False, 
    sql_queries=False
) #Run the actual forecast

In [ ]:
# Pull in most recent timeline
timeline = pd.read_excel(os.path.join(homey, 'RegularForecast.xlsx'), sheet_name='Timeline')

In [ ]:
# Query part costs
myresults = connecttest.create_connection(sqlPath, 'AvgCostQuery.txt')
myexcel = connecttest.makeexcelsheet(myresults)
connecttest.save_workbook(myexcel, rawDataPath, 'AvgCosts.xlsx')

In [ ]:
# Get part costs into a dataFrame
costpath = os.path.join(rawDataPath, 'AvgCosts.xlsx')
avgCost = pd.read_excel(costpath) #Opens and puts the data into a dataframe
avgCost = avgCost.sort_values(by='PART', ascending=True) #Sort the data by part

In [ ]:
# Merge part costs onto timeline
timeline = pd.merge(timeline.copy(), avgCost.copy(), how='left', on='PART')

In [ ]:
# Calculate total costs
timeline['TotalCost'] = timeline['QTYREMAINING'] * timeline['AvgCost']

# Grab list of imaginary builds and create build number reference
newbuilds = pd.read_excel(os.path.join(AdditionalInfoPath, 'PartsToBuild.xlsx'), sheetname='Sheet1')
newbuilds.reset_index(inplace=True)
newbuilds['buildIndex'] = (newbuilds['index'] * -1) -1
newbuilds.drop('index', axis=1, inplace=True)

# Create columns to be filled by the next loop.
newbuilds['Total_Purchase'] = 0
newbuilds['Material_Cost'] = 0

# This dataFrame will hold cost issues for all the imaginary builds
missingCost = pd.DataFrame()

""" This loop calculates and updates the total purchasing need for each imaginary build
    and it also finds the total raw material cost. """
for index, row in newbuilds.iterrows():
    imagBuild = row['buildIndex']
    thisBuild = timeline[timeline['GRANDPARENT'] == imagBuild].copy()
    purchDF = thisBuild[thisBuild['ORDERTYPE'] == 'Purchase'].copy()
    totPurchDF = purchDF[purchDF['TotalCost'].notnull()].copy()
    newbuilds.set_value(index, 'Total_Purchase', purchDF['TotalCost'].sum())
    # Now adding the total raw good cost of "Buy" items to show total amount in movement
    buyItemsDF = thisBuild[thisBuild['Make/Buy'] == 'Buy'].copy()
    rawGoodsDF = buyItemsDF[buyItemsDF['ORDERTYPE'] == 'Raw Good']
    newbuilds.set_value(index, 'Material_Cost', -1*(rawGoodsDF['TotalCost'].sum()))
    # missingCost records parts that have 0 cost or are completely missing cost
    noCostDF = rawGoodsDF[(rawGoodsDF['TotalCost'].isnull()) | (rawGoodsDF['TotalCost'] == 0)].copy() # ... it's so nice ...
    noCostDF['Build'] = newbuilds.get_value(index, 'Part')
    missingCost = missingCost.copy().append(noCostDF.copy())

# Create a column representing cost in material either in inventory currently or already on order
newbuilds['Current_Cost_Used'] = newbuilds['Material_Cost'] - newbuilds['Total_Purchase']

# Generic labor estimate based solely on total material cost of the build
newbuilds['Labor_Estimate'] = newbuilds['Material_Cost'] * 0.2

# Cost including materials and labor estimate
newbuilds['Total_Cost'] = newbuilds['Material_Cost'] + newbuilds['Labor_Estimate']

timeline.to_clipboard()

newbuilds.to_clipboard()

missingCost.to_clipboard()

newbuilds.sum()

missingCost

timeline.to_clipboard(excel=True, sep="comma")

to_clipboard() stopped working.

In [ ]:
writer = pd.ExcelWriter(os.path.join(homey, 'BuildCostResults.xlsx'))  # Creates a test excel file
timeline.to_excel(writer, 'timeline', index=False)  # Fills the test excel with the whole timeline
# newbuilds.to_excel(writer, 'newbuilds', index=False)
# missingCost.to_excel(writer, 'missingCost', index=False)
writer.save()

This section is checking inventory cost for backlog or something

In [ ]:
# Query part costs
myresults = connecttest.create_connection(sqlPath, 'AssetQuery.txt')
myexcel = connecttest.makeexcelsheet(myresults)
connecttest.save_workbook(myexcel, rawDataPath, 'Asset.xlsx')

In [ ]:
assetpath = os.path.join(rawDataPath, 'Asset.xlsx')
asset = pd.read_excel(assetpath) #Opens and puts the data into a dataframe
asset = asset.sort_values(by='PART', ascending=True) #Sort the data by part

In [ ]:
assettypepath = os.path.join(rawDataPath, 'AssetByType.xlsx')
typeAsset = pd.read_excel(assettypepath) #Opens and puts the data into a dataframe

In [ ]:
asset = pd.merge(asset.copy(), typeAsset.copy(), how='left', on='ASSET')

In [ ]:
# theoretically this is the total cost of everything that we don't have but need to fill the backlog.  Some currently issued POs would count toward this.
timeline['TotalCost'][timeline['ORDERTYPE'] == 'Purchase'].copy().sum()

In [ ]:
footy = timeline.copy()

In [ ]:
startInv = footy.drop_duplicates('PART', keep='first').copy()

In [ ]:
endInv = footy.drop_duplicates('PART', keep='last').copy()

In [ ]:
startInv['STARTINV'] = startInv['INV'].copy()
endInv['ENDINV'] = endInv['INV'].copy()

In [ ]:
doots = pd.merge(startInv.copy(), endInv[['PART','ENDINV']].copy(), how='left', on='PART')

In [ ]:
doots = doots[['PART','STARTINV','ENDINV']].copy()

In [ ]:
invpath = os.path.join(rawDataPath, 'INVs.xlsx')
origInv = pd.read_excel(invpath)

In [ ]:
origInv.rename({'INV':'STARTINV'}, axis=1, inplace=True)

In [ ]:
origInv['ENDINV'] = origInv['STARTINV'].copy()

In [ ]:
doots = doots.copy().append(origInv.copy(), sort=False)

In [ ]:
doots.drop_duplicates('PART', keep='first', inplace=True)

In [ ]:
doots['INVUSED'] = doots['STARTINV'].copy() - doots['ENDINV'].copy()

In [ ]:
doots = pd.merge(doots.copy(), avgCost.copy(), how='left', on='PART')

In [ ]:
doots['USEDTOTALCOST'] = doots['INVUSED'].copy() * doots['AvgCost'].copy()

In [ ]:
doots['STARTTOTALCOST'] = doots['STARTINV'].copy() * doots['AvgCost'].copy()

In [ ]:
doots['ENDTOTALCOST'] = doots['ENDINV'].copy() * doots['AvgCost'].copy()

In [ ]:
doots = pd.merge(doots.copy(), asset.copy(), how='left', on='PART')

In [ ]:
writer = pd.ExcelWriter(os.path.join(homey, 'doots.xlsx'))  # Creates a test excel file
doots.to_excel(writer, 'timeline', index=False)  # Fills the test excel with the whole timeline
writer.save()

In [ ]:
mfgDoots = doots[doots['TYPE'] == 'MFG'].copy()

In [ ]:
integDoots = doots[doots['TYPE'] == 'INTEG'].copy()

In [ ]:
mfgDoots['USEDTOTALCOST'][mfgDoots['INVUSED'] > 0].sum()

In [ ]:
mfgDoots['STARTTOTALCOST'].sum()

In [ ]:
mfgDoots['ENDTOTALCOST'].sum()

In [ ]:
mfgDoots['USEDTOTALCOST'].sum() + mfgDoots['ENDTOTALCOST'].sum()

In [ ]:
mfgDoots['USEDTOTALCOST'].sum()

In [ ]:
# Query part costs
myresults = connecttest.create_connection(sqlPath, 'InvCostLayerQuery.txt')
myexcel = connecttest.makeexcelsheet(myresults)
connecttest.save_workbook(myexcel, rawDataPath, 'InvCostLayer.xlsx')

In [ ]:
costLayerPath = os.path.join(rawDataPath, 'InvCostLayer.xlsx')
costLayer = pd.read_excel(costLayerPath) #Opens and puts the data into a dataframe
costLayer = costLayer.sort_values(by='PART', ascending=True) #Sort the data by part

In [ ]:
inv2018 = costLayer[['PART','2018INV']].copy().groupby('PART').sum().reset_index()

In [ ]:
statime = pd.merge(doots.copy(), inv2018.copy(), how='left', on='PART')

In [ ]:
statime.reset_index(drop=True, inplace=True)

In [ ]:
statime.fillna(0, inplace=True)

In [ ]:
statime['STALESTART'] = 0
statime['FRESHSTART'] = 0
statime['STALEUSED'] = 0
statime['FRESHUSED'] = 0
statime['STALEEND'] = 0
statime['FRESHEND'] = 0

In [ ]:
statime

In [ ]:
for index in statime.index:
    if statime['2018INV'].at[index] >= statime['ENDINV'].at[index]:
        statime['FRESHEND'].at[index] = statime['ENDINV'].at[index]
        statime['STALEEND'].at[index] = 0
    else:
        statime['FRESHEND'].at[index] = statime['2018INV'].at[index]
        statime['STALEEND'].at[index] = statime['ENDINV'].at[index] - statime['2018INV'].at[index]
    if statime['2018INV'].at[index] >= statime['STARTINV'].at[index]:
        statime['FRESHSTART'].at[index] = statime['STARTINV'].at[index]
        statime['STALESTART'].at[index] = 0
    else:
        statime['FRESHSTART'].at[index] = statime['2018INV'].at[index]
        statime['STALESTART'].at[index] = statime['STARTINV'].at[index] - statime['2018INV'].at[index]
    statime['STALEUSED'].at[index] = statime['STALESTART'].at[index] - statime['STALEEND'].at[index]
    statime['FRESHUSED'].at[index] = statime['FRESHSTART'].at[index] - statime['FRESHEND'].at[index]

In [ ]:
partspath = os.path.join(rawDataPath, 'Parts.xlsx')
parts = pd.read_excel(partspath) #Opens and puts the data into a dataframe
parts = parts.sort_values(by='PART', ascending=True) #Sort the data by part

In [ ]:
statime = pd.merge(statime.copy(), parts[['PART','Make/Buy']].copy(), how='left', on='PART')

In [ ]:
desc = pd.read_excel(os.path.join(rawDataPath, 'Descs.xlsx'), header=0)

In [ ]:
statime = pd.merge(statime.copy(), desc.copy(), how='left', on='PART')

In [ ]:
statime['STALESTARTVAL'] = statime['STALESTART'].copy() * statime['AvgCost'].copy()
statime['FRESHSTARTVAL'] = statime['FRESHSTART'].copy() * statime['AvgCost'].copy()
statime['STALEUSEDVAL'] = statime['STALEUSED'].copy() * statime['AvgCost'].copy()
statime['FRESHUSEDVAL'] = statime['FRESHUSED'].copy() * statime['AvgCost'].copy()
statime['STALEENDVAL'] = statime['STALEEND'].copy() * statime['AvgCost'].copy()
statime['FRESHENDVAL'] = statime['FRESHEND'].copy() * statime['AvgCost'].copy()

In [ ]:
headerStatime = ['PART','DESCRIPTION','STARTINV','STARTTOTALCOST','ENDINV','ENDTOTALCOST','INVUSED','USEDTOTALCOST','STALESTART','STALESTARTVAL','FRESHSTART','FRESHSTARTVAL','STALEUSED','STALEUSEDVAL','FRESHUSED','FRESHUSEDVAL','STALEEND','STALEENDVAL','FRESHEND','FRESHENDVAL','AvgCost','ASSET','TYPE','2018INV','Make/Buy']

In [ ]:
statime = statime[headerStatime].copy()

In [ ]:
writer = pd.ExcelWriter(os.path.join(homey, 'statime.xlsx'))  # Creates a test excel file
statime.to_excel(writer, 'timeline', index=False)  # Fills the test excel with the whole timeline
writer.save()

In [ ]:
statime